In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [2]:
from pathlib import Path

import numpy as np
import torch

from tqdm import tqdm

from src.net import OptSatGNN
from src.utils import load_from_wandb

In [9]:
run_id = '64o2jqgs'

net = OptSatGNN(
    n_h_feats=256,
    single_conv_for_both_passes=True,
    n_passes=3,
    conv1='SAGEConv',
    # conv1_kwargs=dict(),
    conv2=None,
    conv2_kwargs=dict(),
)

net = load_from_wandb(net, run_id, 'sat-gnn')

In [13]:
run_id = 'ae645j8u'

net = OptSatGNN(
    n_h_feats=64,
    single_conv_for_both_passes=False,
    n_passes=3,
    conv1='SAGEConv',
    # conv1_kwargs=dict(),
    conv2=None,
    conv2_kwargs=dict(),
)

net = load_from_wandb(net, run_id, 'sat-gnn')

In [14]:
run_id = 'uiwvukto'

net = OptSatGNN(
    n_h_feats=64,
    single_conv_for_both_passes=True,
    n_passes=3,
    conv1='SAGEConv',
    # conv1_kwargs=dict(),
    conv2=None,
    conv2_kwargs=dict(),
)

net = load_from_wandb(net, run_id, 'sat-gnn')

In [20]:
Y = list()
P_hat = list()
for i in tqdm(list(range(len(test_data)))):
    g, y = test_data[i]

    with torch.no_grad():
        p_hat = torch.sigmoid(net(g)).item()
    
    Y.append(y)
    P_hat.append(p_hat)

  0%|          | 0/6000 [00:00<?, ?it/s]

100%|██████████| 6000/6000 [00:52<00:00, 113.32it/s]


In [23]:
P_hat = np.array(P_hat).reshape(6,1000)
Y = np.array(Y).reshape(6,1000)

In [30]:
Y_hat = (P_hat > 0.5).astype(Y.dtype)
Y_hat

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [38]:
H = (Y_hat == Y).astype(int)

H = {
    20: H[:2],
    22: H[2:4],
    24: H[4:],
}

H_opt = {s: H[s][:,:500] for s in H.keys()}
H_dirty = {s: H[s][:,500:-250] for s in H.keys()}
H_random = {s: H[s][:,-250:] for s in H.keys()}

for size in H.keys():
    print(size)
    print(H[size].sum() / H[size].size)
    print(H_opt[size].sum() / H_opt[size].size)
    print(H_dirty[size].sum() / H_dirty[size].size)
    print(H_random[size].sum() / H_random[size].size)

20
1.0
1.0
1.0
1.0
22
0.875
1.0
0.5
1.0
24
1.0
1.0
1.0
1.0
